In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from textblob import TextBlob

In [ ]:
data=pd.read_csv('/Users/cherucole/Desktop/tweets.csv',encoding='latin-1')

In [ ]:
cols=list(pd.read_csv('/Users/cherucole/Desktop/tweets.csv',encoding='latin-1'))

In [ ]:
data2=pd.read_csv('/Users/cherucole/Desktop/tweets.csv',encoding='latin-1',usecols = ['text'])

In [ ]:
data.describe()

In [ ]:
data.describe(include='O')

In [ ]:
data.columns.tolist()

In [ ]:
data.dtypes

In [ ]:
data[['text','tweet_location']]

In [ ]:
data['tweet_location'].isnull().count()

In [ ]:
len(data)# total values

In [ ]:
location_data=data['tweet_location'].count() #non empty values

In [ ]:
no_location_data=len(data) - data['tweet_location'].count()


In [ ]:
location_data

In [ ]:
no_location_data


## Natural Language Processing

In [ ]:
import nltk
import ftfy
from ftfy import *
from nltk.tokenize import sent_tokenize,word_tokenize,TweetTokenizer
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
data.head()

In [ ]:
y=data['tweet_location']

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data2, y, test_size=0.2)


In [ ]:
print('shapes of the X data are:train {}, test {}'.format(X_train.shape,X_test.shape))

In [ ]:
print('shapes of the y data are:train {}, test {}'.format(y_train.shape,y_test.shape))

### Data Pre- Processing and  Analysis

In [ ]:
a=y_train.isnull().value_counts()

In [ ]:
a

In [ ]:
X_train['text'].shape


In [ ]:
blob=TextBlob()

In [ ]:
for x in X_train['text']:
    blob=TextBlob(x)
    print(blob.sentiment.polarity)

## Advanced Text Processing

#### 1.Word Count

In [ ]:
data['word_count']=data['text'].apply(lambda x:len(str(x).split(" ")))

In [ ]:
data.shape

#### 2.Number of Characters

In [ ]:
data['char_count']=data['text'].str.len()

#### 3.Average Word Length

In [ ]:

def avg_word(sentence):
    words=sentence.split()
    return sum(len(word) for word in words)/len(words)


data['avg_word']=data['text'].apply(lambda x:avg_word(x))

In [ ]:
from nltk.corpus import stopwords


#### 4.Number of Stopwords

In [ ]:
stop=stopwords.words('english')

In [ ]:
data['stopwords']=data['text'].apply(lambda x: len([x for x in x.split() if x in stop]))

#### 4. Number of hashtags(special characters)

In [ ]:
data['hashtags']=data['text'].apply(lambda x:len([x for x in x.split()if x.startswith('#')]))

In [ ]:
train2=train['text'].to_string()

In [ ]:
train3=fix_text_segment(train2,normalization=('NFKC'))

In [ ]:
tknzr = TweetTokenizer()

In [ ]:
 train4=tknzr.tokenize(train2)

In [ ]:
train4

In [ ]:
stop_words=set(stopwords.words('english'))
tokens=[x for x in train4 if not x in stop_words]
print(tokens)

## Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
porter=PorterStemmer()
stems=[]
for i in tokens:
    stems.append(porter.stem(i))
print(stems)    

In [ ]:
from textblob import Word

In [ ]:
data['tweet_lematized']=data['text'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))

In [ ]:
data['tweet_lematized'][1]

In [ ]:
TextBlob(X_train['tweet_lematized'][1]).ngrams(2)

In [ ]:
data['tweet_lematized'].shape


## Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
tfidf=TfidfVectorizer(max_features=None,lowercase=True,analyzer='word',stop_words='english',ngram_range=(1,1))

In [ ]:
 data['data_vect']=tfidf.fit_transform(data['tweet_lematized'])


In [ ]:
data.head(5)


In [ ]:
X_train['text'][:5].apply(lambda x: TextBlob(x).sentiment)

In [ ]:
X_train['tweet_lematized'][:5].apply(lambda x:TextBlob(x).sentiment)

In [ ]:
data['sentiments_polarity']=data['tweet_lematized'].apply(lambda x:TextBlob(x).sentiment)

In [ ]:
data[['tweet_lematized','sentiments_polarity']]

In [ ]:
# def place(location):
#     if location == 'clcncl':
#         return 0
#     else:
#         return 1
# train['labels']=train['tweet_location'].apply(lambda x:place(x))

In [ ]:
# def checkLocation(location):
#     if location == 'NaN':
#         return 'This is not location'
# train['labels']=train['tweet_location'].apply(lambda x:checkLocation(x))

In [ ]:
data['location2']=data['tweet_location'].apply(lambda x:pd.isnull(x))

In [ ]:
def place(location2):
    if location2==True:
        return 0
    else:
        return 1
data['labels']=data['location2'].apply(lambda x:place(x))

In [ ]:
data[['tweet_location','location2','labels']]

## MACHINE LEARNING

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [ ]:
y=data['labels']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2)


In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
y_test.shape

In [ ]:
y_train.shape

In [ ]:
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
